<h3>Imports and setup</h3>

In [36]:
from pathlib import Path
import os
import tifffile
import czifile
from tqdm import tqdm
import napari
import pandas as pd
import numpy as np


<h3>Image data scraping</h3>

In [37]:
# Define the directory containing the raw .czi files and annotations
directory_path = Path("./apeer_annotations/")

# Initialize an empty list to store the .czi and ome.tiff annotation filepaths
czi_images = []
annotations = []

# Iterate through the files in the directory
for file_path in directory_path.glob('*.czi'):
    # Append the file to the corresponding group
    czi_images.append(str(file_path))

# Iterate through the files in the directory
for file_path in directory_path.glob('*.tiff'):
    # Append the file to the corresponding group
    annotations.append(str(file_path))


<h3>Transforming .czi files into .tiff</h3>

In [57]:
# Define output directory for .tiff converted .czi files  
processed_czi_dir = "./processed_bf_organoids/train_images"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_czi_dir)
    print(f"Output folder created: {processed_czi_dir}")
except FileExistsError:
    print(f"Output folder already exists: {processed_czi_dir}")
    
for filepath in tqdm(czi_images):
    # Get the filename without the extension
    filename = Path(filepath).stem

    # Read the .czi file and remove singleton dimensions
    img = czifile.imread(filepath)
    img = img.squeeze()

    # Save the image as a regular .tiff file under processed_bf_organoids/train_images
    tifffile.imwrite(os.path.join(processed_czi_dir, f"{filename}.tiff"), img)


Output folder created: ./processed_bf_organoids/train_images


100%|██████████| 119/119 [00:01<00:00, 80.41it/s]


<h3>Rename annotations and create a copy under preprocessed_annotations</h3>

In [58]:
# Define output directory for renamed annotations
processed_annotations = "./preprocessed_annotations"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_annotations)
    print(f"Output folder created: {processed_annotations}")
except FileExistsError:
    print(f"Output folder already exists: {processed_annotations}")
    
for filepath in tqdm(annotations):
    # Get the filename without the extension
    filename = Path(filepath).stem
    # Remove _finalprediction.ome so it matches the original image names
    filename = filename.replace("_finalprediction.ome", "")
    
    # Read the ome.tiff file and remove singleton dimensions, ignore ome
    img = tifffile.imread(filepath, is_ome=False)
    
    # Save the image as a regular .tiff file under preprocessed_annotations
    tifffile.imwrite(os.path.join(processed_annotations, f"{filename}.tiff"), img)

Output folder created: ./preprocessed_annotations


100%|██████████| 119/119 [00:01<00:00, 79.30it/s]


<h3>Process dead_organoid annotations into dead_org_train_masks</h3>

In [81]:
# Define output directory for dead_organoid annotations
processed_dead_annotations = "./processed_bf_organoids/train_masks/dead"

# Check if the output folder exists, create it if not
try:
    os.makedirs(processed_dead_annotations)
    print(f"Output folder created: {processed_dead_annotations}")
except FileExistsError:
    print(f"Output folder already exists: {processed_dead_annotations}")

# Extract the channel dimension containing the dead organoids annotations
dead_organoids = img[0,:,:]

# Binarize the array
dead_organoids = np.where(dead_organoids !=0, 255, 0)

# Convert the array to uint8
dead_organoids = dead_organoids.astype(np.uint8)

# Store the resulting array as a .tiff file under processed_dead_annotations
tifffile.imwrite(os.path.join(processed_dead_annotations, f"MIN_P1A01.tiff"), dead_organoids)





Output folder already exists: ./processed_bf_organoids/train_masks/dead


In [53]:
# Read measurements_default.csv containing each object_id's class_name
df = pd.read_csv("./apeer_annotations/measurements_default.csv")

In [65]:
img = tifffile.imread("./preprocessed_annotations/MIN_P1A01.tiff", is_ome=False)
viewer = napari.Viewer(ndisplay=2)
viewer.add_labels(img, name="MIN_P1A01")

<Labels layer 'MIN_P1A01' at 0x1663df17220>